In [16]:
import datetime

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution1D, TimeDistributed, Conv2D, MaxPooling1D
from keras.layers.recurrent import LSTM, GRU
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, CSVLogger
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import *
from keras.optimizers import Adam, Nadam

%matplotlib

Using matplotlib backend: Qt5Agg


## Import Tick Data and Create 5min RTH Bars

In [2]:
tick_data = pd.read_feather('../data/processed/ES_tick.feather')
tick_data = tick_data[tick_data['date'] > '2017-07-29']
#Create Index from date column
tick_data.index = tick_data['date']
tick_data.drop(labels=['date'],axis=1,inplace=True)
tick_data.tail()

,last,bid,ask,volume
date,,,,
2018-01-25 19:54:52.612000-05:00,2844.0,2843.75,2844.0,1
2018-01-25 19:54:52.615000-05:00,2844.0,2843.75,2844.0,1
2018-01-25 19:54:54.157000-05:00,2844.0,2843.75,2844.0,1
2018-01-25 19:54:54.157000-05:00,2844.0,2843.75,2844.0,1
2018-01-25 19:54:55.332000-05:00,2844.0,2843.75,2844.0,2


In [3]:
#Resample to get 5min bars
five_min_data = pd.DataFrame(
    tick_data['last'].resample('5Min', loffset=datetime.timedelta(minutes=5)).ohlc())

import pandas_market_calendars as mcal
#We hack the NYSE Calendar extending the close until 4:15
class CMERTHCalendar(mcal.exchange_calendar_nyse.NYSEExchangeCalendar):
    @property
    def close_time(self):
        return datetime.time(16, 15)
    
#Create RTH Calendar
nyse = CMERTHCalendar()
schedule = nyse.schedule(start_date=five_min_data.index.min(), 
                         end_date=five_min_data.index.max())

#Filter out those bars that occur during RTH
five_min_data['dates'] = pd.to_datetime(five_min_data.index.to_datetime().date)
five_min_data['valid_date'] = five_min_data['dates'].isin(schedule.index)
five_min_data['valid_time'] = False
during_rth = five_min_data['valid_date'] & \
        (five_min_data.index > schedule.loc[five_min_data['dates'],'market_open']) & \
        (five_min_data.index <= schedule.loc[five_min_data['dates'],'market_close'])
five_min_data.loc[during_rth, 'valid_time'] = True
five_min_data = five_min_data[five_min_data['valid_time'] == True]
five_min_data.drop(['dates','valid_date','valid_time'], axis=1, inplace=True)

#Add ema
five_min_data['ema'] = five_min_data['close'].ewm(span=20, min_periods=20).mean()

#Reset index
five_min_data.reset_index(inplace=True)

five_min_data[81:].head()

,date,open,high,low,close,ema
81,2017-08-01 09:35:00-04:00,2475.50,2476.00,2472.50,2473.50,2470.908594
82,2017-08-01 09:40:00-04:00,2473.50,2474.00,2471.50,2472.50,2471.060194
83,2017-08-01 09:45:00-04:00,2472.25,2473.25,2471.75,2473.00,2471.244978
84,2017-08-01 09:50:00-04:00,2473.00,2473.25,2472.00,2472.75,2471.388343
85,2017-08-01 09:55:00-04:00,2472.75,2473.00,2471.25,2471.25,2471.375165


In [83]:
#Add column for number of seconds elapsed in trading day
five_min_data['sec'] = (five_min_data['date'].values 
                        - five_min_data['date'].values.astype('datetime64[D]')) / np.timedelta64(1,'s')

#Calculate sin & cos time
#24hr time is a cyclical continuous feature
seconds_in_day = 24*60*60
five_min_data['sin_time'] = np.sin(2*np.pi*five_min_data['sec']/seconds_in_day)
five_min_data['cos_time'] = np.cos(2*np.pi*five_min_data['sec']/seconds_in_day)

five_min_data.drop('sec', axis=1, inplace=True)
five_min_data.head()

,date,open,high,low,close,ema,sin_time,cos_time
0,2017-07-31 09:35:00-04:00,2474.75,2475.75,2474.00,2475.50,NaN,-0.402747,-0.915311
1,2017-07-31 09:40:00-04:00,2475.25,2476.00,2473.75,2475.50,NaN,-0.422618,-0.906308
2,2017-07-31 09:45:00-04:00,2475.75,2475.75,2474.50,2474.75,NaN,-0.442289,-0.896873
3,2017-07-31 09:50:00-04:00,2474.50,2475.00,2473.50,2473.75,NaN,-0.461749,-0.887011
4,2017-07-31 09:55:00-04:00,2474.00,2474.25,2472.75,2472.75,NaN,-0.480989,-0.876727


## Create Test / Train Datasets

In [85]:
data = five_min_data[81:]

openp = data['open'].tolist()
highp = data['high'].tolist()
lowp = data['low'].tolist()
closep = data['close'].tolist()
emap = data['ema'].tolist()
sin_time = data['sin_time'].tolist()
cos_time = data['cos_time'].tolist()

In [177]:
WINDOW = 64 #Number of bars in a trading day
EMB_SIZE = 7
STEP = 1
FORECAST = 1

X, Y = [], []
for i in range(0, len(data), STEP):
    try:
        o = openp[i:i+WINDOW]
        h = highp[i:i+WINDOW]
        l = lowp[i:i+WINDOW]
        c = closep[i:i+WINDOW]
        e = emap[i:i+WINDOW]
        ct = cos_time[i:i+WINDOW]
        st = sin_time[i:i+WINDOW]

        #o = (np.array(o) - np.mean(o)) / np.std(o)
        #h = (np.array(h) - np.mean(h)) / np.std(h)
        #l = (np.array(l) - np.mean(l)) / np.std(l)
        #c = (np.array(c) - np.mean(c)) / np.std(c)
        #e = (np.array(e) - np.mean(e)) / np.std(e)
        
        o = np.array(o) / c[-1] 
        h = np.array(h) / c[-1]
        l = np.array(l) / c[-1]
        e = np.array(e) / c[-1]
        c = np.array(c) / c[-1]

        x_i = closep[i:i+WINDOW]
        y_i = closep[(i+WINDOW-1)+FORECAST]  

        last_close = x_i[-1]
        next_close = y_i

        if last_close >= next_close:
            y_i = [1, 0]
        else:
            y_i = [0, 1] 
        
        x_i = np.column_stack((o, h, l, c, e, ct, st))

    except Exception as e:
        #e.throw()
        break

    X.append(x_i)
    Y.append(y_i)

In [178]:
# Let's split into train and test sets
# Train Set will be from 8/1/17 through 12/31/17, Test Set 1/1/17 - 1/25/17
p = 8547 #Manual split for now
#p=8448
X, Y = np.array(X), np.array(Y)
X_train = X[0:p]
Y_train = Y[0:p]
X_test = X[p:]
Y_test = Y[p:]

#We may want to shuffle the training data -- will look into this later
def shuffle_in_unison(a, b):
    # courtsey http://stackoverflow.com/users/190280/josh-bleecher-snyder
    assert len(a) == len(b)
    shuffled_a = np.empty(a.shape, dtype=a.dtype)
    shuffled_b = np.empty(b.shape, dtype=b.dtype)
    permutation = np.random.permutation(len(a))
    for old_index, new_index in enumerate(permutation):
        shuffled_a[new_index] = a[old_index]
        shuffled_b[new_index] = b[old_index]
    return shuffled_a, shuffled_b

#X_train, Y_train = shuffle_in_unison(X_train, Y_train)

# Not sure why this is needed, but we apply it anyway
X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1], EMB_SIZE))
X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1], EMB_SIZE))
X_train[-1]

array([[[ 9.92938772e-01,  9.93589148e-01,  9.92938772e-01,
          9.93403326e-01,  9.95877377e-01,  7.51839807e-01,
         -6.59345815e-01],
        [ 9.98606337e-01,  9.98699247e-01,  9.96933940e-01,
          9.97026851e-01,  9.95986851e-01, -7.79884483e-01,
         -6.25923472e-01],
        [ 9.97026851e-01,  9.98234693e-01,  9.96655208e-01,
          9.98141782e-01,  9.96192083e-01, -7.66044443e-01,
         -6.42787610e-01],
        [ 9.98141782e-01,  9.98606337e-01,  9.98048871e-01,
          9.98141782e-01,  9.96377768e-01, -7.51839807e-01,
         -6.59345815e-01],
        [ 9.98234693e-01,  9.98234693e-01,  9.97491406e-01,
          9.97677228e-01,  9.96501526e-01, -7.37277337e-01,
         -6.75590208e-01],
        [ 9.97584317e-01,  9.97955960e-01,  9.97305584e-01,
          9.97770138e-01,  9.96622346e-01, -7.22363962e-01,
         -6.91513056e-01],
        [ 9.97770138e-01,  9.98885069e-01,  9.97491406e-01,
          9.98792158e-01,  9.96828995e-01, -7.07106781e-01

## Train CNN Model

In [192]:
model = Sequential()

model.add(
    TimeDistributed(
        Conv2D(32, (7, 7), padding='same', strides=2),
        input_shape=(None, 540, 960, 2)))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_37 (TimeDis (None, None, 270, 480, 32 3168      
Total params: 3,168
Trainable params: 3,168
Non-trainable params: 0
_________________________________________________________________


In [179]:
model = Sequential()
model.add(TimeDistributed(Convolution1D(filters=16,
                                        kernel_size=4,
                                        padding='same'),
                          input_shape = (None, WINDOW, EMB_SIZE)))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(LeakyReLU()))
model.add(TimeDistributed(Dropout(0.5)))

model.add(TimeDistributed(Convolution1D(filters=8,
                        kernel_size=4,
                        padding='same')))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(LeakyReLU()))
model.add(TimeDistributed(Dropout(0.5)))

model.add(TimeDistributed(Flatten()))

model.add(TimeDistributed((Dense(32))))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(LeakyReLU()))

model.add(LSTM(32, dropout=0, stateful=False))
model.add(Dropout(0))

model.add(Dense(2))
model.add(Activation('softmax'))
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_25 (TimeDis (None, None, 64, 16)      464       
_________________________________________________________________
time_distributed_26 (TimeDis (None, None, 64, 16)      64        
_________________________________________________________________
time_distributed_27 (TimeDis (None, None, 64, 16)      0         
_________________________________________________________________
time_distributed_28 (TimeDis (None, None, 64, 16)      0         
_________________________________________________________________
time_distributed_29 (TimeDis (None, None, 64, 8)       520       
_________________________________________________________________
time_distributed_30 (TimeDis (None, None, 64, 8)       32        
_________________________________________________________________
time_distributed_31 (TimeDis (None, None, 64, 8)       0         
__________

In [175]:
model = Sequential()
model.add(Convolution1D(input_shape = (WINDOW, EMB_SIZE),
                        filters=16,
                        kernel_size=4,
                        padding='same'))
model.add(BatchNormalization())
model.add(LeakyReLU())
#model.add(MaxPooling1D(strides=1))
model.add(Dropout(0.5))

model.add(Convolution1D(filters=8,
                        kernel_size=4,
                        padding='same'))
model.add(BatchNormalization())
model.add(LeakyReLU())
#model.add(MaxPooling1D(strides=1))
model.add(Dropout(0.5))

model.add(Flatten())

model.add(Dense(32))
model.add(BatchNormalization())
model.add(LeakyReLU())


model.add(Dense(2))
model.add(Activation('softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_65 (Conv1D)           (None, 64, 16)            464       
_________________________________________________________________
batch_normalization_94 (Batc (None, 64, 16)            64        
_________________________________________________________________
leaky_re_lu_97 (LeakyReLU)   (None, 64, 16)            0         
_________________________________________________________________
dropout_65 (Dropout)         (None, 64, 16)            0         
_________________________________________________________________
conv1d_66 (Conv1D)           (None, 64, 8)             520       
_________________________________________________________________
batch_normalization_95 (Batc (None, 64, 8)             32        
_________________________________________________________________
leaky_re_lu_98 (LeakyReLU)   (None, 64, 8)             0         
__________

In [180]:
opt = Nadam(lr=0.0001)

reduce_lr = ReduceLROnPlateau(monitor='val_acc', factor=0.9, patience=30, min_lr=0.000001, verbose=1)
checkpointer = ModelCheckpoint(filepath="model.hdf5", verbose=1, save_best_only=True)


model.compile(optimizer=opt, 
              loss='categorical_crossentropy',
              metrics=['accuracy'])

history = model.fit(X_train, Y_train, 
          epochs = 100, 
          batch_size = 128, 
          verbose=1, 
          validation_data=(X_test, Y_test),
          callbacks=[reduce_lr, checkpointer],
          shuffle='batch')

Train on 8547 samples, validate on 1313 samples
Epoch 1/100
8547/8547 [==============================] - 7s 803us/step - loss: 0.6914 - acc: 0.5243 - val_loss: 0.6922 - val_acc: 0.5225

Epoch 00001: val_loss improved from inf to 0.69224, saving model to model.hdf5
Epoch 2/100
8547/8547 [==============================] - 1s 144us/step - loss: 0.6907 - acc: 0.5318 - val_loss: 0.6926 - val_acc: 0.5179

Epoch 00002: val_loss did not improve from 0.69224
Epoch 3/100
8547/8547 [==============================] - 1s 144us/step - loss: 0.6896 - acc: 0.5424 - val_loss: 0.6928 - val_acc: 0.5225

Epoch 00003: val_loss did not improve from 0.69224
Epoch 4/100
8547/8547 [==============================] - 1s 147us/step - loss: 0.6894 - acc: 0.5383 - val_loss: 0.6927 - val_acc: 0.5225

Epoch 00004: val_loss did not improve from 0.69224
Epoch 5/100
8547/8547 [==============================] - 1s 152us/step - loss: 0.6879 - acc: 0.5510 - val_loss: 0.6928 - val_acc: 0.5187

Epoch 00005: val_loss did not 

Epoch 43/100
8547/8547 [==============================] - 1s 155us/step - loss: 0.6859 - acc: 0.5545 - val_loss: 0.6915 - val_acc: 0.5232

Epoch 00043: val_loss improved from 0.69160 to 0.69149, saving model to model.hdf5
Epoch 44/100
8547/8547 [==============================] - 1s 148us/step - loss: 0.6864 - acc: 0.5545 - val_loss: 0.6913 - val_acc: 0.5240

Epoch 00044: val_loss improved from 0.69149 to 0.69133, saving model to model.hdf5
Epoch 45/100
8547/8547 [==============================] - 1s 149us/step - loss: 0.6860 - acc: 0.5559 - val_loss: 0.6914 - val_acc: 0.5278

Epoch 00045: val_loss did not improve from 0.69133
Epoch 46/100
8547/8547 [==============================] - 1s 155us/step - loss: 0.6856 - acc: 0.5552 - val_loss: 0.6918 - val_acc: 0.5240

Epoch 00046: val_loss did not improve from 0.69133
Epoch 47/100
8547/8547 [==============================] - 1s 155us/step - loss: 0.6852 - acc: 0.5560 - val_loss: 0.6914 - val_acc: 0.5278

Epoch 00047: val_loss did not improve

8547/8547 [==============================] - 1s 141us/step - loss: 0.6848 - acc: 0.5554 - val_loss: 0.6918 - val_acc: 0.5270

Epoch 00085: val_loss did not improve from 0.69115
Epoch 86/100
8547/8547 [==============================] - 1s 140us/step - loss: 0.6849 - acc: 0.5570 - val_loss: 0.6917 - val_acc: 0.5270

Epoch 00086: val_loss did not improve from 0.69115
Epoch 87/100
8547/8547 [==============================] - 1s 142us/step - loss: 0.6842 - acc: 0.5573 - val_loss: 0.6918 - val_acc: 0.5263

Epoch 00087: val_loss did not improve from 0.69115
Epoch 88/100
8547/8547 [==============================] - 1s 143us/step - loss: 0.6849 - acc: 0.5559 - val_loss: 0.6917 - val_acc: 0.5278

Epoch 00088: val_loss did not improve from 0.69115
Epoch 89/100
8547/8547 [==============================] - 1s 145us/step - loss: 0.6846 - acc: 0.5575 - val_loss: 0.6918 - val_acc: 0.5263

Epoch 00089: val_loss did not improve from 0.69115
Epoch 90/100
8547/8547 [==============================] - 1s 14

In [181]:
plt.figure()
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='best')
plt.show()

plt.figure()
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='best')
plt.show()

In [173]:
from sklearn.metrics import confusion_matrix

model.load_weights("model.hdf5")
pred = model.predict(np.array(X_test), batch_size=128)

C = confusion_matrix([np.argmax(y) for y in Y_test], [np.argmax(y) for y in pred])

print (C / C.astype(np.float).sum(axis=1)[:, None])

[[1. 0.]
 [1. 0.]]


In [41]:
pred[2]

array([0.5830708 , 0.41692916], dtype=float32)

In [170]:
df = pd.DataFrame(np.concatenate((pred, Y_test), axis=1))
df[df[0]>.6].head()
df.sort_values(1,axis=0).head(20)

,0,1,2,3
1198,0.585188,0.414812,1.0,0.0
874,0.585003,0.414997,1.0,0.0
469,0.584993,0.415007,1.0,0.0
307,0.584852,0.415148,1.0,0.0
388,0.584818,0.415182,1.0,0.0
145,0.584813,0.415187,1.0,0.0
955,0.584811,0.415189,1.0,0.0
550,0.584795,0.415205,0.0,1.0
631,0.584754,0.415246,0.0,1.0
712,0.584741,0.415259,1.0,0.0


In [174]:
C

array([[812,   0],
       [501,   0]])

In [45]:
C / C.astype(np.float).sum(axis=1)[:, None]

array([[0.83673469, 0.15855573, 0.00470958],
       [0.78915663, 0.20481928, 0.0060241 ],
       [0.86335404, 0.13043478, 0.00621118]])

In [69]:
probs = Y_train.sum(axis=0) / Y_train.shape[0]
probs

array([0.55539956, 0.44460044])

In [68]:
pred

array([[0.4436597 , 0.5563404 ],
       [0.5040323 , 0.49596766],
       [0.5359964 , 0.4640036 ],
       ...,
       [0.57311577, 0.4268842 ],
       [0.5362818 , 0.46371824],
       [0.5139088 , 0.48609126]], dtype=float32)

In [120]:
s = np.random.binomial(1, probs[1], pred.shape[0])
s

array([1, 0, 1, ..., 0, 0, 1])

In [121]:
C1 = confusion_matrix([np.argmax(y) for y in Y_test], s)
print (C1 / C1.astype(np.float).sum(axis=1)[:, None])

[[0.58421851 0.41578149]
 [0.59026688 0.40973312]]


In [122]:
([np.argmax(y) for y in Y_test] == s).sum() / pred.shape[0]

0.4984567901234568